<a href="https://colab.research.google.com/github/RicardColomOrero/easymoney/blob/culebra/Tabla_customer_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tabla customer products - Culebra

##Imports

In [128]:
import sys
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

from google.colab import files
from google.colab import drive


from sklearn import model_selection
import xgboost as xgb

from scipy import stats
from datetime import datetime

import random

In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
DATA_PATH = '/content/drive/MyDrive/Master_DS/TFM  Data Science/datasets_TFM + diccionario/'
FILE = 'customer_products.csv'

In [131]:
cust_products = pd.read_csv(DATA_PATH + FILE)

## Data Understanding. EDA

### Visualización directa de datos

In [132]:
cust_products.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
pk_cid,1375586,1050611,1050612,1050613,1050614
pk_partition,2018-01,2018-01,2018-01,2018-01,2018-01
short_term_deposit,0,0,0,1,0
loans,0,0,0,0,0
mortgage,0,0,0,0,0
funds,0,0,0,0,0
securities,0,0,0,0,0
long_term_deposit,0,0,0,0,0
em_account_pp,0,0,0,0,0


### Tamaño del dataset

In [133]:
cust_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   pk_cid              int64  
 2   pk_partition        object 
 3   short_term_deposit  int64  
 4   loans               int64  
 5   mortgage            int64  
 6   funds               int64  
 7   securities          int64  
 8   long_term_deposit   int64  
 9   em_account_pp       int64  
 10  credit_card         int64  
 11  payroll             float64
 12  pension_plan        float64
 13  payroll_account     int64  
 14  emc_account         int64  
 15  debit_card          int64  
 16  em_account_p        int64  
 17  em_acount           int64  
dtypes: float64(2), int64(15), object(1)
memory usage: 818.9+ MB


- pk_cid =	Identificador de cliente
- pk_partition =	Identificador de Partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes.
- credit_card =	Tenencia de tarjetas de crédito
- debit_card =	Tenencia de tarjetas de débito
- em_account_p =	Tenencia de Cuenta easyMoney+
- em_account_pp =	Tenencia de Cuenta easyMoney++
- em_acount =	Tenencia de Cuenta easyMoney
- emc_account =	Tenencia de Cuenta easyMoney Crypto
- funds =	Tenencia de Fondos de inversión
- loans =	Tenencia de Préstamos
- long_term_deposit =	Tenencia de Depósitos a largo plazo
- mortgage =	Tenencia de hipoteca
- payroll =	Tenencia de Domiciliaciones 
- payroll_account =	Tenencia de Cuenta bonificada por domiciliaciones
- pension_plan =	Tenencia de Plan de pensiones
- securities =	Tenencia de Valores
- short_term_deposit =	Tenencia de Short-term deposits

In [134]:
#Se elimina la columna Unnamed:0

In [135]:
del cust_products['Unnamed: 0']

In [136]:
cust_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 17 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  pension_plan        float64
 12  payroll_account     int64  
 13  emc_account         int64  
 14  debit_card          int64  
 15  em_account_p        int64  
 16  em_acount           int64  
dtypes: float64(2), int64(14), object(1)
memory usage: 773.4+ MB


In [137]:
cust_products.head().T

,0,1,2,3,4
pk_cid,1375586,1050611,1050612,1050613,1050614
pk_partition,2018-01,2018-01,2018-01,2018-01,2018-01
short_term_deposit,0,0,0,1,0
loans,0,0,0,0,0
mortgage,0,0,0,0,0
funds,0,0,0,0,0
securities,0,0,0,0,0
long_term_deposit,0,0,0,0,0
em_account_pp,0,0,0,0,0
credit_card,0,0,0,0,0


###2.1.3 Tipos de atributos disponibles , 2.1.4 Estadisticos descriptivos y 2.1.5. Numero de valores nulos

In [138]:
cust_products.isnull().sum()

pk_cid                 0
pk_partition           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
em_account_pp          0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0
dtype: int64

In [139]:
cust_products.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
pk_cid,5962924.0,1.234930e+06,162302.043492,15891.0,1112532.0,1231097.0,1352339.0,1553689.0
short_term_deposit,5962924.0,2.581619e-03,0.050744,0.0,0.0,0.0,0.0,1.0
loans,5962924.0,7.848498e-05,0.008859,0.0,0.0,0.0,0.0,1.0
mortgage,5962924.0,5.433576e-05,0.007371,0.0,0.0,0.0,0.0,1.0
funds,5962924.0,3.370662e-03,0.057959,0.0,0.0,0.0,0.0,1.0
securities,5962924.0,3.711770e-03,0.060811,0.0,0.0,0.0,0.0,1.0
long_term_deposit,5962924.0,1.688098e-02,0.128826,0.0,0.0,0.0,0.0,1.0
em_account_pp,5962924.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0
credit_card,5962924.0,1.188444e-02,0.108366,0.0,0.0,0.0,0.0,1.0
payroll,5962863.0,3.451144e-02,0.182539,0.0,0.0,0.0,0.0,1.0


Por una parte tenemos el pk_cid, y por otra el resto de variables numéricas.

- A la variable pk_cid la vamos a llamar customer_ID, y la transformaremos a object.
- Ese resto de variables son boleanas, vamos a alinear su dtype a tipo int, de todos modos más tarde el OHE las pasará automáticamente a boleanas.
- la variable "em_account_pp" no tiene datos, la eliminamos

In [140]:
cust_products = cust_products.rename(columns={'pk_cid': 'customer_ID'})

In [141]:
cust_products['customer_ID'] = cust_products['customer_ID'].astype('object')

In [142]:
variables_to_int = cust_products.select_dtypes(include=['float']).columns #metemos en el saco las decimales

In [143]:
#cust_products[variables_to_bool] = cust_products[variables_to_bool].applymap(int) LO HAREMOS UNA VEZ TENGAMOS LOS NULOS IMPUTADOS

In [144]:
del cust_products['em_account_pp']

In [145]:
cust_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   customer_ID         object 
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   credit_card         int64  
 9   payroll             float64
 10  pension_plan        float64
 11  payroll_account     int64  
 12  emc_account         int64  
 13  debit_card          int64  
 14  em_account_p        int64  
 15  em_acount           int64  
dtypes: float64(2), int64(12), object(2)
memory usage: 727.9+ MB


In [146]:
cust_products.describe(include=object).T

,count,unique,top,freq
customer_ID,5962924,456373,1375586,17
pk_partition,5962924,17,2019-05,442995


Con pk_partition lo que haremos será:

- Primero alinear el nombre a partition_ID
- Después cambiarla a tipo date yyyy/mm
- No vamos a dividir en mes y año ya que no nos va a dar información extra

In [147]:
cust_products = cust_products.rename(columns={'pk_partition': 'partition_ID'})

In [148]:
cust_products['partition_ID'] = pd.to_datetime(cust_products['partition_ID'], format='%Y-%m', dayfirst=True).dt.to_period('M')

In [149]:
cust_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 16 columns):
 #   Column              Dtype    
---  ------              -----    
 0   customer_ID         object   
 1   partition_ID        period[M]
 2   short_term_deposit  int64    
 3   loans               int64    
 4   mortgage            int64    
 5   funds               int64    
 6   securities          int64    
 7   long_term_deposit   int64    
 8   credit_card         int64    
 9   payroll             float64  
 10  pension_plan        float64  
 11  payroll_account     int64    
 12  emc_account         int64    
 13  debit_card          int64    
 14  em_account_p        int64    
 15  em_acount           int64    
dtypes: float64(2), int64(12), object(1), period[M](1)
memory usage: 727.9+ MB


In [150]:
cust_products.head()

,customer_ID,partition_ID,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01,1,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [104]:
cust_products.isnull().sum()

customer_ID            0
partition_ID           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0
dtype: int64

Como vamos a hacer un modelo de clasificación, los nulos se rellenan con valores extremos, en este caso como es una booleana le vamos a poner un -1.

**¡OJO!** Si en estas variables tuvieramos más del 50% de algún producto de Easy Money como 1, es decir más del 50% de los clientes adquieren el producto, el valor extremo que le imputamos sería un 2 en lugar de un -1, ya que el modelo podrá precisar mejor.

**IMPORTANTE!** **La única variable donde es recomendable eliminar los nulos es en la variable TARGET**. 

In [152]:
cust_products.dropna(subset = ['pension_plan'], inplace = True)

In [153]:
cust_products.isnull().sum()

customer_ID           0
partition_ID          0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
credit_card           0
payroll               0
pension_plan          0
payroll_account       0
emc_account           0
debit_card            0
em_account_p          0
em_acount             0
dtype: int64

In [154]:
cust_products[variables_to_int] = cust_products[variables_to_int].applymap(int)

In [155]:
cust_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962863 entries, 0 to 5962923
Data columns (total 16 columns):
 #   Column              Dtype    
---  ------              -----    
 0   customer_ID         object   
 1   partition_ID        period[M]
 2   short_term_deposit  int64    
 3   loans               int64    
 4   mortgage            int64    
 5   funds               int64    
 6   securities          int64    
 7   long_term_deposit   int64    
 8   credit_card         int64    
 9   payroll             int64    
 10  pension_plan        int64    
 11  payroll_account     int64    
 12  emc_account         int64    
 13  debit_card          int64    
 14  em_account_p        int64    
 15  em_acount           int64    
dtypes: int64(14), object(1), period[M](1)
memory usage: 773.4+ MB


In [109]:
# DUDAS

# No he establecido como índice el customer_ID por lo siguiente. Cuando poner una columna como índice, ¿la perdemos como columna a la hora de analizar?
# Se puede plantear que customer_ID sea numérica si asumimos que es una forma de medir la antiguedad del cliente? A menor número ID es un cliente anterior en el tiempo




In [156]:
#¿Cuántos clientes únicos tenemos?
cust_products['customer_ID'].nunique()

456373

In [157]:
cust_products[(cust_products['partition_ID']=='2019-05')].shape

(442995, 16)

In [ ]:
for col in cust_products.columns:
    print(f"Value counts for column '{col}':")
    print(cust_products[col].value_counts(normalize=True))

### Análisis del TARGET

In [158]:
TARGET = 'pension_plan'

In [159]:
cust_products[TARGET].describe()

count    5.962863e+06
mean     3.652641e-02
std      1.875960e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: pension_plan, dtype: float64

In [160]:
cust_products[TARGET].value_counts()

0    5745061
1     217802
Name: pension_plan, dtype: int64

In [162]:
cust_products[TARGET].value_counts(normalize = True)

0    0.963474
1    0.036526
Name: pension_plan, dtype: float64

In [165]:
cust_products[(cust_products[TARGET]==1)]['payroll'].value_counts()

1    205787
0     12015
Name: payroll, dtype: int64

In [171]:
int_cols = cust_products.select_dtypes(include='int').columns
for col in int_cols:
  print(f"Value counts for column '{col}' when TARGET is 1:")
  print(cust_products[(cust_products[TARGET]==1)][col].value_counts(normalize = True))

Value counts for column 'short_term_deposit' when TARGET is 1:
0    0.999082
1    0.000918
Name: short_term_deposit, dtype: float64
Value counts for column 'loans' when TARGET is 1:
0    0.999137
1    0.000863
Name: loans, dtype: float64
Value counts for column 'mortgage' when TARGET is 1:
0    0.998949
1    0.001051
Name: mortgage, dtype: float64
Value counts for column 'funds' when TARGET is 1:
0    0.989679
1    0.010321
Name: funds, dtype: float64
Value counts for column 'securities' when TARGET is 1:
0    0.98343
1    0.01657
Name: securities, dtype: float64
Value counts for column 'long_term_deposit' when TARGET is 1:
0    0.969279
1    0.030721
Name: long_term_deposit, dtype: float64
Value counts for column 'credit_card' when TARGET is 1:
0    0.860249
1    0.139751
Name: credit_card, dtype: float64
Value counts for column 'payroll' when TARGET is 1:
1    0.944835
0    0.055165
Name: payroll, dtype: float64
Value counts for column 'pension_plan' when TARGET is 1:
1    1.0
Name: 